<a href="https://colab.research.google.com/github/Dax1805/cs6200-documents/blob/main/PageRank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PageRank

In this exercise, you will compute PageRank on a collection of 469,235 web sites using the iterative version of the PageRank algorithm described in class for sparse graphs.  As we discussed in class, this avoids performing cubic-time operations on a graph with a large number of nodes.

Furthermore, you should take advantage of the _sparsity_ of the original hyperlinks, even though "teleportation" or random jumps connect every note to every other node.  As long as the maximum degree of the original link graph is much less than the number of nodes, you should be able to keep each iteration's runtime less than quadratic in the number of nodes.

Consider the following directed graph:

![A directed link graph](https://www.khoury.northeastern.edu/home/dasmith/courses/cs6200//pagerank.jpg)

We can represent this graph as a collection of nodes, here, ordered pairs of node index and node name:

In [1]:
small_nodes = [(0, 'A'),
              (1, 'B'),
              (2, 'C'),
              (3, 'D'),
              (4, 'E'),
              (5, 'F')]

and a collection of directed links, i.e., ordered pairs from source to target:

In [2]:
small_edges = [
  (0, 1),
  (0, 2),
  (0, 5),
  (1, 2),
  (1, 3),
  (1, 4),
  (1, 5),
  (2, 3),
  (2, 4),
  (3, 0),
  (3, 2),
  (3, 4),
  (3, 5),
  (4, 0),
  (5, 0),
  (5, 1),
  (5, 4)
]

We use integer identifiers for the nodes for efficiency. In most real-world collections of hyperlinks, unlike this example, not every page will have in-links, nor will every page have out-links.

## First Implementation and Test

\[10 points\] Implement the iterative PageRank algorithm we discussed in class. Test your code on the six-node example using the input representation given above.  Be sure that your code handles pages that have no in-links or out-links properly.  (You may wish to test on a few such examples.) In later parts of this assignment, depending on how you store the data, it may be convenient to use iterators rather than storing the data in memory.

In [6]:
# TODO: Implement PageRank, given nodes and edges, to start with a uniform
# distribution over nodes, run a fixed number of iterations, and
# return a distribution over nodes.

def page_rank_fixed_iter(nodes, edges, iterations=10, d=0.85):
    n = len(nodes)
    pr = {node_id: 1.0 / n for node_id, _ in nodes}

    outlinks = {node_id: [] for node_id, _ in nodes}
    out_degree = {node_id: 0 for node_id, _ in nodes}
    for src, dst in edges:
        outlinks[src].append(dst)
        out_degree[src] += 1

    for _ in range(iterations):
        new_pr = {node_id: (1 - d) / n for node_id, _ in nodes}
        for node_id, rank in pr.items():
            if out_degree[node_id]:
                share = rank / out_degree[node_id]
                for target in outlinks[node_id]:
                    new_pr[target] += d * share
            else:
                share = rank / n
                for target in new_pr:
                    new_pr[target] += d * share
        pr = new_pr

    return pr

# Output PageRank on the toy graph at various points.
# Make sure your output has node number, name, and PageRank value.
print(page_rank_fixed_iter(small_nodes, small_edges, 1))
print(page_rank_fixed_iter(small_nodes, small_edges, 10))
print(page_rank_fixed_iter(small_nodes, small_edges, 100))

{0: 0.24930555555555556, 1: 0.11944444444444445, 2: 0.14305555555555555, 3: 0.13125, 4: 0.2138888888888889, 5: 0.14305555555555555}
{0: 0.25203637602817186, 1: 0.1393065091825108, 2: 0.15129376593475072, 3: 0.11896297277689881, 4: 0.18710661014291716, 5: 0.15129376593475072}
{0: 0.2521271053751956, 1: 0.1393061853185386, 2: 0.1513064898667053, 3: 0.11890782257353921, 4: 0.1870459069993161, 5: 0.1513064898667053}


## PageRank on Web Crawl Data

\[20 points\] Download and unpack a list of `.edu` websites and the links among them from the [Common Crawl](https://commoncrawl.org/2017/05/hostgraph-2017-feb-mar-apr-crawls/) open-source web crawl. For the sake of brevity, the data record links among websites, not web pages. The information for nodes and links is the same as the toy example above.

In [7]:
# If you're running on a machine (e.g., Windows) that doesn't have wget or gzip,
# feel free to comment this out and use a different set of commands to load
# the data.
!wget https://khoury.northeastern.edu/home/dasmith/courses/cs6200/vertices-edu.txt.gz
!gzip -df vertices-edu.txt.gz
!wget https://khoury.northeastern.edu/home/dasmith/courses/cs6200/edges-edu.txt.gz
!gzip -df edges-edu.txt.gz

--2025-02-22 22:06:41--  https://khoury.northeastern.edu/home/dasmith/courses/cs6200/vertices-edu.txt.gz
Resolving khoury.northeastern.edu (khoury.northeastern.edu)... 52.70.229.197
Connecting to khoury.northeastern.edu (khoury.northeastern.edu)|52.70.229.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3703486 (3.5M) [application/x-gzip]
Saving to: ‘vertices-edu.txt.gz’

vertices-edu.txt.gz 100%[===================>]   3.53M  5.49MB/s    in 0.6s    

2025-02-22 22:06:42 (5.49 MB/s) - ‘vertices-edu.txt.gz’ saved [3703486/3703486]

--2025-02-22 22:06:42--  https://khoury.northeastern.edu/home/dasmith/courses/cs6200/edges-edu.txt.gz
Resolving khoury.northeastern.edu (khoury.northeastern.edu)... 52.70.229.197
Connecting to khoury.northeastern.edu (khoury.northeastern.edu)|52.70.229.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12829526 (12M) [application/x-gzip]
Saving to: ‘edges-edu.txt.gz’

edges-edu.txt.gz    100%[===========

There should now be files `vertices-edu.txt` and `edges-edu.txt`.

In [9]:
# TODO: Process the raw data into the same format as the simple graph.
# You may create lists or iterators.

nodes = []
with open('vertices-edu.txt', 'r') as f:
    for line in f:
        parts = line.strip().split()
        if len(parts) < 2:
            continue
        node_id = int(parts[0])
        node_name = " ".join(parts[1:])
        nodes.append((node_id, node_name))

edges = []
with open('edges-edu.txt', 'r') as f:
    for line in f:
        parts = line.strip().split()
        if len(parts) < 2:
            continue
        src = int(parts[0])
        dst = int(parts[1])
        edges.append((src, dst))

print("Total number of nodes:", len(nodes))
print("Total number of edges:", len(edges))

Total number of nodes: 469235
Total number of edges: 3300462


Refine your implementation of PageRank to test for numerical convergence. Specificially, at each iteration, calculate the [perplexity](https://en.wikipedia.org/wiki/Perplexity) of the PageRank distribution, where perplexity is defined as 2 raised to the [Shannon entropy](https://en.wikipedia.org/wiki/Entropy_(information_theory)) of the PageRank distribution, i.e., $2^{H(PR)}$. (Recall the definition of entropy from our discussion of data compression.) The maximum perplexity of a PageRank distribution will therefore be the number of nodes in the graph.

At each iteration, check the _change_ in perplexity. If the change is less than some threshold, you can stop.


In [15]:
# TODO: Implement convergence testing in PageRank
# If you choose, you can share some subroutines with your first version.
# Print the change in perplexity at each iteration.

import math

def page_rank(nodes, edges, threshold=1, d=0.85):
    n = len(nodes)
    pr = {node: 1.0 / n for node, _ in nodes}

    outlinks = {node: [] for node, _ in nodes}
    out_degree = {node: 0 for node, _ in nodes}
    for src, dst in edges:
        outlinks[src].append(dst)
        out_degree[src] += 1

    def perplexity(pr_dist):
        entropy = -sum(p * math.log(p, 2) for p in pr_dist.values() if p > 0)
        return 2 ** entropy

    prev_perp = None
    iter_count = 0

    while True:
        iter_count += 1
        # Total PageRank from sink node
        sink_total = sum(pr[node] for node in pr if out_degree[node] == 0)
        # Base contribution: teleportation plus sink distribution.
        new_pr = {node: (1 - d) / n + d * (sink_total / n) for node, _ in nodes}
        # Distribute PageRank from non-sink nodes.
        for node in pr:
            if out_degree[node]:
                share = pr[node] / out_degree[node]
                for target in outlinks[node]:
                    new_pr[target] += d * share

        pr = new_pr
        cur_perp = perplexity(pr)
        change = abs(cur_perp - prev_perp) if prev_perp is not None else None
        if change is not None:
            print(f"Iteration {iter_count}: Perplexity = {cur_perp:.6f}, Change = {change:.6f}")
            if change < threshold:
                break
        else:
            print(f"Iteration {iter_count}: Perplexity = {cur_perp:.6f}")
        prev_perp = cur_perp

    return pr
# Run until perplexity changes by less than 1
PR = page_rank(nodes, edges, 1)

Iteration 1: Perplexity = 298071.446167
Iteration 2: Perplexity = 255742.814745, Change = 42328.631423
Iteration 3: Perplexity = 234252.272154, Change = 21490.542590
Iteration 4: Perplexity = 225975.955895, Change = 8276.316259
Iteration 5: Perplexity = 221538.999751, Change = 4436.956144
Iteration 6: Perplexity = 219533.964586, Change = 2005.035165
Iteration 7: Perplexity = 218300.922517, Change = 1233.042069
Iteration 8: Perplexity = 217671.939029, Change = 628.983488
Iteration 9: Perplexity = 217238.014611, Change = 433.924418
Iteration 10: Perplexity = 216999.383617, Change = 238.630994
Iteration 11: Perplexity = 216815.233264, Change = 184.150353
Iteration 12: Perplexity = 216708.529683, Change = 106.703581
Iteration 13: Perplexity = 216619.067759, Change = 89.461924
Iteration 14: Perplexity = 216564.521951, Change = 54.545808
Iteration 15: Perplexity = 216516.228208, Change = 48.293743
Iteration 16: Perplexity = 216485.351334, Change = 30.876874
Iteration 17: Perplexity = 216457.

## Link Analysis

\[20 points\] In this final section, you will compute some properties of this web-site graph.

First, consider the _in-link count_ of a website, simply the number of websites pointing to it (including self-links).

In [16]:
# TODO: List the document ID, domain name, and in-link count of the 70 websites with the highest in-link count

in_link_counts = {node_id: 0 for node_id, _ in nodes}
for src, dst in edges:
    in_link_counts[dst] += 1

# Combine node information with the in-link counts.
node_inlinks = [(node_id, domain, in_link_counts[node_id]) for node_id, domain in nodes]

# Sort the nodes in descending order by in-link count.
top_70 = sorted(node_inlinks, key=lambda x: x[2], reverse=True)[:70]

print("Document ID, Domain Name, In-Link Count")
for node_id, domain, count in top_70:
    print(f"{node_id}\t{domain}\t{count}")

Document ID, Domain Name, In-Link Count
185524	edu.mit.web	4388
278032	edu.stanford	4021
244433	edu.purdue.english.owl	3531
140443	edu.indiana	3339
237176	edu.princeton	3251
64587	edu.columbia	3123
465503	edu.yale	2804
418623	edu.utexas	2622
383763	edu.unc	2592
197698	edu.nap	2494
439637	edu.washington	2291
373442	edu.umich	2281
440674	edu.washington.depts	2276
148945	edu.jhu.muse	2255
60975	edu.colorado	2232
449738	edu.wisc	2230
38320	edu.bu	2205
83572	edu.dartmouth	1965
408380	edu.usc	1952
178879	edu.mit	1946
27307	edu.berkeley	1908
233405	edu.pitt	1857
191069	edu.msu	1810
326371	edu.uchicago.press	1763
136464	edu.illinois	1753
93874	edu.educause	1741
56979	edu.cmu.cs	1730
199032	edu.ncsu	1709
36294	edu.brown	1702
202182	edu.nd	1689
68675	edu.cornell	1685
71095	edu.cornell.law	1646
183214	edu.mit.mitpress	1644
215627	edu.nyu	1625
56538	edu.cmu	1583
239378	edu.psu	1541
350412	edu.ufl	1533
120819	edu.harvard	1529
270369	edu.si	1513
107916	edu.gatech	1500
365396	edu.uky	1497
337138	edu.

Then, use the PageRank values compute by your second implementation. Note that some websites will have both a high in-link count and PageRank.

In [17]:
# TODO: List the document ID, domain name, and PageRank of the 70 websites with the highest PageRank.
pr_info = [(node_id, domain, PR[node_id]) for node_id, domain in nodes]

# Sort the list by PageRank
top_70_pr = sorted(pr_info, key=lambda x: x[2], reverse=True)[:70]

print("Document ID, Domain Name, PageRank")
for node_id, domain, pr_val in top_70_pr:
    print(f"{node_id}\t{domain}\t{pr_val:.6f}")

Document ID, Domain Name, PageRank
185524	edu.mit.web	0.000928
465503	edu.yale	0.000701
318278	edu.ucdavis.cas	0.000642
278032	edu.stanford	0.000594
136464	edu.illinois	0.000580
319795	edu.ucdavis.ice.code	0.000547
237176	edu.princeton	0.000502
284517	edu.stanford.web	0.000475
383763	edu.unc	0.000472
64587	edu.columbia	0.000447
449738	edu.wisc	0.000436
60975	edu.colorado	0.000432
346969	edu.ucsf	0.000430
140443	edu.indiana	0.000430
42502	edu.byu.cas	0.000418
334739	edu.uconn	0.000413
14945	edu.arizona	0.000409
244433	edu.purdue.english.owl	0.000403
373442	edu.umich	0.000402
439637	edu.washington	0.000398
408380	edu.usc	0.000391
191069	edu.msu	0.000391
281817	edu.stanford.med	0.000391
418623	edu.utexas	0.000382
239378	edu.psu	0.000379
68675	edu.cornell	0.000374
342997	edu.ucsd	0.000358
153823	edu.kit	0.000358
293521	edu.tamu	0.000357
197698	edu.nap	0.000353
337138	edu.ucop	0.000346
233405	edu.pitt	0.000344
307291	edu.ttu.depts	0.000340
323918	edu.uchicago	0.000337
225417	edu.osu	0.00033

Finally, compute some summary statistics on this dataset.

In [18]:
# TODO: Compute:
# - the proportion of websites with no in-links (i.e., source nodes);

# - the proportion of websites with no out-links (i.e., sink nodes);

# - the proportion of websites whose PageRank is higher than the initial uniform distribution.

num_nodes = len(nodes)

# Compute in-link counts for each node.
in_link_counts = {node_id: 0 for node_id, _ in nodes}
for src, dst in edges:
    in_link_counts[dst] += 1

# Proportion of websites with no in-links
num_no_in_links = sum(1 for node_id, _ in nodes if in_link_counts[node_id] == 0)
prop_no_in_links = num_no_in_links / num_nodes

# Compute out-degree for each node.
out_degree = {node_id: 0 for node_id, _ in nodes}
for src, dst in edges:
    out_degree[src] += 1

# Proportion of websites with no out-links
num_no_out_links = sum(1 for node_id, _ in nodes if out_degree[node_id] == 0)
prop_no_out_links = num_no_out_links / num_nodes

# Proportion of websites whose PageRank is higher than the initial uniform distribution.
uniform_value = 1 / num_nodes
num_above_uniform = sum(1 for node_id, _ in nodes if PR[node_id] > uniform_value)
prop_above_uniform = num_above_uniform / num_nodes

print("Proportion of websites with no in-links (source nodes):", prop_no_in_links)
print("Proportion of websites with no out-links (sink nodes):", prop_no_out_links)
print("Proportion of websites with PageRank higher than the uniform value:", prop_above_uniform)


Proportion of websites with no in-links (source nodes): 0.26153633041013563
Proportion of websites with no out-links (sink nodes): 0.6106641661427643
Proportion of websites with PageRank higher than the uniform value: 0.15514614212494807
